# 3webアプリ作成

目次：https://docs.google.com/document/d/1eDw9PK5Ft0vImu81e7oERB_q-dZvxbFe8bGetZBgEEg/edit

## 3.3 応用実装

#### ④OCRアプリ


>アプリの概要

<img src="pic/img042.png">

デプロイ先：https://ocr-step2-ver6-d89jchryrlb.streamlit.app/

>動作を構成してる部品の紹介（APIやライブラリ）
画像認識外部ツール
- Tesseract-OCR
    - 目的：python環境内でTesseract-OCRを活用して、画像から文字認識をする
    - 参考：https://github.com/tesseract-ocr/tessdoc

ライブラリ
- streamlit
    - 目的：画像分析をするためのUIを表示するためのフロントエンドを生成する
- pyocr
    - 目的：外部ツールTesseract-OCRをpythonで活用するためのライブラリ
- pillow
    - 目的：python内で画像を読み込んで、Tesseract-OCRに画像データを渡すために使う
- asari
    - 目的：文字から簡易的な感情分析をするために使う





>ライブラリのインストール

外部ツール
- Tesseract-OCR
    - Windowsの場合：tesseract-ocr-w64-setup-5.3.1.20230401.exeでインストール
        - インストーラーのDL場所：https://github.com/UB-Mannheim/tesseract/wiki
    - 2015年以前のintelCPU搭載Macの場合：brewコマンドでインストール
        - brew install tesseract
        - brew install tesseract-lang
    - M1チップなどのappleシリコンのMacの場合：condaでインストール
        - conda install -c conda-forge tesseract


ライブラリ
- streamlit
    - pip install streamlit
- pyocr
    - pip install pyocr
- Pillow
    - pip install Pillow
- asari
    - pip install asari



>ライブラリの公式ドキュメントの見方や使い方を説明

ライブラリ

- streamlit
    - インストール方法はget startページで確認
    - 参考：https://docs.streamlit.io/library/get-started/installation
    - st.titleなどはコンポーネントページから確認出来る
    - 参考：https://streamlit.io/components
- pyocr
    - pyocr.get_available_tools()で使用できるOCRツールを検知してengine.image_to_string(画像情報)でocrを使った画像分析を開始
    - 本来は公式ですが、公式に使い方の情報ないので、エンジニアが共有しているコミュニティを活用させていただきます。
    - コード参考：https://qiita.com/ku_a_i/items/93fdbd75edacb34ec610
    - なにも説明がない公式：https://pypi.org/project/pyocr/
- Pillow
    - Image.open(画像ファイルの場所)でpython環境内に画像ファイルを読み込む
    - 参考：https://pillow.readthedocs.io/en/stable/
- asari
    - sonar.ping(text=感情分析したい文字)で感情分析の結果が得られます。
    - 参考：https://pypi.org/project/asari/

>それぞれの機能をjupyterで動く最小限を作る

以下がjuptyterにて動く最低限のアプリの流れです。

1. 作るアプリの全体のコードを把握
1. それぞれ機能を解説
    1. 外部OCRを活用するpyocrの仕様
    1. 画像解析
    1. 簡易的な感情分析

1.全体のコード

使うアプリの全体像は下記のようになります。  
これはweb上では動きませんが、手元では動作するアプリです。

以下のコードの動作の流れ
1. 拡張機能であるライブラリをインポート
1. 外部OCRの準備
    1. ライブラリpyocrに外部ツールであるtesseractがどこにあるかを指定
    1. ライブラリpyocrが認識している外部ツールを設定し、engineに代入
1. 外部OCRを使って画像解析
    1. engine.image_to_stringで画像から文字読み取り
1. 画像から文字読み取った文字から感情分析
    1. 拡張機能であるライブラリをインポート
    1. sonar.pingで文字から簡易的な感情分析


In [3]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from PIL import Image # 画像扱うための機能をインポート
import pyocr # 外部OCRを扱うための機能をインポート
import platform # OSの判別するために、プラットフォームを読み込む機能をインポート

# それぞれのOSにインストールされるtesseractの場所を指定
# ※講義のためにmacでもwindowsでも動くように指定しています。
# ※macでエラーが出る場合は、ターミナルで「which tesseract」を実行して、出力されたパスをpyocr.tesseract.TESSERACT_CMDに設定してください。
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"

print("文字認識アプリ") # タイトル表示

print("moji.png") # 画像分析する画像を表示

# OCRエンジンを取得
engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

# 画像の文字を読み込む
# engine.image_to_string(開いた画像,画像認識する言語)で画像分析開始し、分析結果をtxtに代入
txt = engine.image_to_string(Image.open("moji.png"), lang="jpn")
print(txt) # 分析結果を表示

print("感情分析の結果") # 案内表示
from asari.api import Sonar # 文字から感情分析する機能をインポート
sonar = Sonar() # Sonar()をsonarに代入
res = sonar.ping(text=txt) # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入
print(res["classes"]) # res["classes"]に結果が変えて来るので、これを表示

文字認識アプリ
moji.png


TesseractError: (1, b'Error opening data file /usr/local/share/tessdata/jpn.traineddata\nPlease make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory.\nFailed loading language \'jpn\'\nTesseract couldn\'t load any languages!\nCould not initialize tesseract.\n')

2.それぞれ機能を解説

2.1 外部OCRを活用するpyocrの仕様

pyocrは外部ツールとしてtesseractを活用するため、必ずインストールとツールの位置の指定が必要です。  

ツールの位置の指定はpyocr.tesseract.TESSERACT_CMDを使います。  
また、指定した場所からpyocrが読み込んだツールを.get_available_toolsで呼び出すことができます。

pyocr.get_available_tools()に何もなければ、インストールがうまくいっていないので、  
以下のコードのように、必ずmodule 'pyocr.tesseract'が出力されるようにしてください。

In [19]:
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"

engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
print(engines)

# 次のコードがエラーにならないように、ツールを選択してengineを定義しておく。
engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

[<module 'pyocr.tesseract' from 'c:\\Users\\user\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pyocr\\tesseract.py'>]


2.2画像解析

画像解析は以下のコードでできます。  
engine.image_to_string(Image.open(ここに画像のファイル名), lang=ここに言語)

注意点として、言語コードがjaではなく、jpnになっています。

In [20]:
engine.image_to_string(Image.open("moji.png"), lang="jpn")

'寿限無 寿限無 五却のすりきれ\n\n海砂利水魚の水行末 雲来未 風来未\n\n食う寝るところに住むところ\n\nやぶら小路のぶら小路\n\nパイボポパイポ パイポのシューリンガン\n\nシューリンガンのクーリンダイ\n\nクーリンダイのポンポコナーのポンポコピーの長久命の長助'

2.3簡易的な感情分析

文字からの感情分析は以下のコードでできます。  
Sonar().ping(text=ここに文字列)

結果はjsonとして返ってきます。  
この講座では、この結果が見やすいので、そのまま活用します。

In [24]:
Sonar().ping(text="おはようございます") # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入

{'text': 'おはようございます',
 'top_class': 'positive',
 'classes': [{'class_name': 'negative', 'confidence': 0.009979760274291039},
  {'class_name': 'positive', 'confidence': 0.9900202751159668}]}

以上がアプリの動作部分です。

>streamlitに実装する場合の使う機能

今はユーザーがブラウザ上で選択したりなど、想定していませんが、  
選択肢の表示などのコンポーネントを紹介します。

※streamlitはjupyterでは動作せず、実行ファイルでのみ動作します

タイトルは  
st.title("ここにタイトル")

ただの表示は  
st.write("ここに表示したい文字")

ただし、再読み込みしない限り、この表示された文字は変更されません。

画像の表示は  
st.image("ここにファイル名")

選択肢は  st.selectbox("ここに表示したい項目名",ここに配列のデータ)  
でできます。

ファイルアップロードする機能は  
st.file_uploader("ここに表示したい文字",type=ここに許可する拡張子の配列)  
でできます。

以上の機能を使ってwebアプリにしていきます。

>（演習）streamlitに実装しましょう！

手元で動いたアプリをstreamlitで動くようにします。  
この演習では必ず実行ファイルにコーディングしてください。

※streamlit run ファイル名で実行する時は必ず実行ファイルで実行してください。

実装の流れ
1. 固定である画像ファイルをアップロードされた画像ファイルに変更
    1. アップローダー設置
    1. アップロードされたら画像解析のコードが実行されるようにする
1. 画像解析の認識の言語指定を変更できるようにする
    1. 言語候補を変数へ
    1. セレクトボックスで選択できるようにする
1. ターミナルに出力していた文字などをブラウザに出力

以下の手元で動いたコードをstreamlitで動くようにします。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from PIL import Image # 画像扱うための機能をインポート
import pyocr # 外部OCRを扱うための機能をインポート
import platform # OSの判別するために、プラットフォームを読み込む機能をインポート

# それぞれのOSにインストールされるtesseractの場所を指定
# ※講義のためにmacでもwindowsでも動くように指定しています。
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"

print("文字認識アプリ") # タイトル表示

print("moji.png") # 画像分析する画像を表示

# OCRエンジンを取得
engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

# 画像の文字を読み込む
# engine.image_to_string(開いた画像,画像認識する言語)で画像分析開始し、分析結果をtxtに代入
txt = engine.image_to_string(Image.open("moji.png"), lang="jpn")
print(txt) # 分析結果を表示

print("感情分析の結果") # 案内表示
from asari.api import Sonar # 文字から感情分析する機能をインポート
sonar = Sonar() # Sonar()をsonarに代入
res = sonar.ping(text=txt) # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入
print(res["classes"]) # res["classes"]に結果が変えて来るので、これを表示

1.固定である画像ファイルをアップロードされた画像ファイルに変更

1.1アップローダー設置

読み込みが固定だったmoji.pngをアップローダーからの音声ファイルに変更します。

ファイルアップロードする機能は  
st.file_uploader("ここに表示したい文字",type=ここに許可する拡張子の配列)  
でできます。

今回は許可する拡張子の配列をpngとjpgにします。

In [ ]:
# tesseractが認識できるpngとjpgだけを許可するアップローダーの設置
file_upload = st.file_uploader("ここに音声認識したファイルをアップロードしてください。",type=["png","jpg"])

これで、アップロードされたファイルがfile_uploadに代入されます。

1.2アップロードされたら画像解析のコードが実行されるようにする

手元で動くコードは、無条件にファイルの画像分析をするコードを実行してしまうので、  
これをif文を使ってアップロードされた時だけ、画像分析をするコードを実行するようにします。

st.file_uploaderはなにもアップロードされていない場合、なにも存在しないNoneという値になるので、  
file_upload != Noneという条件を使います。

if文を使った条件は以下のようになります。

In [ ]:
# tesseractが認識できるpngとjpgだけを許可するアップローダーの設置
file_upload = st.file_uploader("ここに音声認識したファイルをアップロードしてください。",type=["png","jpg"])

# ファイルアップロードされた場合、file_uploadがNoneではなくなる
if (file_upload !=None):
    print("ここに画像分析をするコード")

アップローダーと実行条件を手元のコードに反映させると以下のようになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from PIL import Image # 画像扱うための機能をインポート
import pyocr # 外部OCRを扱うための機能をインポート
import platform # OSの判別するために、プラットフォームを読み込む機能をインポート

# それぞれのOSにインストールされるtesseractの場所を指定
# ※講義のためにmacでもwindowsでも動くように指定しています。
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"

print("文字認識アプリ") # タイトル表示



# OCRエンジンを取得
engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

# tesseractが認識できるpngとjpgだけを許可するアップローダーの設置
file_upload = st.file_uploader("ここに音声認識したファイルをアップロードしてください。",type=["png","jpg"])
print(file_upload) # 画像分析する画像を表示
if (file_upload !=None):
    # 画像の文字を読み込む
    # engine.image_to_string(開いた画像,画像認識する言語)で画像分析開始し、分析結果をtxtに代入
    txt = engine.image_to_string(Image.open(file_upload), lang="jpn")
    print(txt) # 分析結果を表示

    print("感情分析の結果") # 案内表示
    from asari.api import Sonar # 文字から感情分析する機能をインポート
    sonar = Sonar() # Sonar()をsonarに代入
    res = sonar.ping(text=txt) # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入
    print(res["classes"]) # res["classes"]に結果が変えて来るので、これを表示

2.画像分析の言語指定を変更できるようにする

画像分析が日本語である"jpn"で固定なので、これをセレクトボックスで選択し、反映できるようにします。

2.1言語候補を変数へ

"jpn"を変数にします。

変数set_language_listに言語コードの辞書型配列を作ります。  
ここでは日本語と英語にしておきます。

英語の言語コードはengです。

In [ ]:
# 画像読み込みのための言語と言語のコードを変換するリストを設定
set_language_list = {
    "日本語" :"jpn",
    "英語" :"eng",
}

セレクトボックスで日本語または英語を選択し、set_language_listで言語コードに変換することで、  
画像分析に必要な言語コードを変数化します。

日本語文字は.keys()で取得できるので、これを活用します。

In [25]:
set_language_list.keys()

dict_keys(['日本語', '英語'])

2.2セレクトボックスで選択できるようにする

この２つの選択肢をセレクトボックスに適応します。

セレクトボックスは  st.selectbox("ここに表示したい項目名",ここに配列のデータ)でできます。  
この選ばれた選択肢をset_languageに代入します。

set_language = st.selectbox("音声認識する言語を選んでください。",set_language_list.keys())

これによって、ユーザーに見える選択肢は、set_languageである['日本語', '英語']から取得し、   
コード実行側では、 set_language_list[set_language]とすることで、言語コードを指定します。

まとめると以下のようなコードになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from PIL import Image # 画像扱うための機能をインポート
import pyocr # 外部OCRを扱うための機能をインポート
import platform # OSの判別するために、プラットフォームを読み込む機能をインポート

# それぞれのOSにインストールされるtesseractの場所を指定
# ※講義のためにmacでもwindowsでも動くように指定しています。
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"


# 画像読み込みのための言語と言語のコードを変換するリストを設定
set_language_list = {
    "日本語" :"jpn",
    "英語" :"eng",
}


print("文字認識アプリ") # タイトル表示

set_language = st.selectbox("音声認識する言語を選んでください。",set_language_list.keys()) # 言語選択のためのリスト



# OCRエンジンを取得
engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

# tesseractが認識できるpngとjpgだけを許可するアップローダーの設置
file_upload = st.file_uploader("ここに音声認識したファイルをアップロードしてください。",type=["png","jpg"])
print(file_upload)  # 画像分析する画像を表示
if (file_upload !=None):
    # 画像の文字を読み込む
    # engine.image_to_string(開いた画像,画像認識する言語)で画像分析開始し、分析結果をtxtに代入
    txt = engine.image_to_string(Image.open(file_upload), lang=set_language_list[set_language])
    print(txt) # 分析結果を表示

    print("感情分析の結果") # 案内表示
    from asari.api import Sonar # 文字から感情分析する機能をインポート
    sonar = Sonar() # Sonar()をsonarに代入
    res = sonar.ping(text=txt) # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入
    print(res["classes"]) # res["classes"]に結果が変えて来るので、これを表示

3.ターミナルに出力していた文字などをブラウザに出力

最後にprintでターミナルで出力していた  
タイトルや文字や音声を以下の４つのコードを使ってブラウザに出力します。

- st.title
- st.write
- st.image

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
from PIL import Image # 画像扱うための機能をインポート
import pyocr # 外部OCRを扱うための機能をインポート
import platform # OSの判別するために、プラットフォームを読み込む機能をインポート

# それぞれのOSにインストールされるtesseractの場所を指定
if platform.system() == "Windows":
    pyocr.tesseract.TESSERACT_CMD = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
else:
    pyocr.tesseract.TESSERACT_CMD = r"/usr/local/bin/tesseract"

# 画像読み込みのための言語と言語のコードを変換するリストを設定
set_language_list = {
    "日本語" :"jpn",
    "英語" :"eng",
}

st.title("文字認識アプリ") # タイトル表示
set_language = st.selectbox("音声認識する言語を選んでください。",set_language_list.keys()) # 言語選択のためのリスト
file_upload = st.file_uploader("ここに音声認識したファイルをアップロードしてください。",type=["png","jpg"]) # tesseractが認識できるpngとjpgだけを許可するアップローダーの設置

# アップロードされたらfile_uploadがNoneではなくなるので、実行される
if (file_upload !=None):

    st.image(file_upload) # 画像分析する画像を表示

    # OCRエンジンを取得
    engines = pyocr.get_available_tools() # pyocrが認識できるOCR外部ツールを検知
    engine = engines[0] # ツールを複数選択して、エラーにならないように１つだけ選択

    # 画像の文字を読み込む
    # engine.image_to_string(開いた画像,画像認識する言語)で画像分析開始し、分析結果をtxtに代入
    txt = engine.image_to_string(Image.open(file_upload), lang=set_language_list[set_language])
    st.write(txt) # 分析結果を表示

    st.write("感情分析の結果") # 案内表示
    from asari.api import Sonar # 文字から感情分析する機能をインポート
    sonar = Sonar() # Sonar()をsonarに代入
    res = sonar.ping(text=txt) # sonar.ping(text=分析したい文字)で感情分析リクエストし、結果をresに代入
    st.write(res["classes"]) # res["classes"]に結果が変えて来るので、これを表示